In [5]:
import awkward as ak
import uproot
import hist as hs
import numpy as np

from coffea.nanoevents import NanoEventsFactory, BaseSchema, NanoAODSchema, TreeMakerSchema, PHYSLITESchema
from coffea.nanoevents.methods import vector

In [2]:
signaldomain     = 'root://cmsxrootd.fnal.gov//store/group/phys_exotica/delayedjets/displacedJetMuonAnalyzer/csc/V1p17/MC_Fall18/v2/v3/normalized/BToKPhi_MuonGenFilter_mPhi1p0_ctau1000_1pb_weighted.root'
signalfile = uproot.open(signaldomain)

backgrounddomain = 'root://cmsxrootd.fnal.gov//store/user/christiw/displacedJetMuonAnalyzer/csc/V1p171/Data2018_UL/v3/normalized/ParkingBPH4_Run2018A_goodLumi.root'
backgroundfile = uproot.open(backgrounddomain)

In [3]:
signalevents = NanoEventsFactory.from_root(
    signalfile,
    treepath='MuonSystem',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()


backgroundevents = NanoEventsFactory.from_root(
    backgroundfile,
    treepath='MuonSystem',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()

In [51]:
signalevents = NanoEventsFactory.from_root(
    signalfile,
    treepath='MuonSystem',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()

def muonCutter(events, cut): # i hate this.
    events.muonE = events.muonE[cut]
    events.muonPt = events.muonPt[cut]
    events.muonEta = events.muonEta[cut]
    events.muonPhi = events.muonPhi[cut]
    events.muonPdgId = events.muonPdgId[cut]
    events.muonDZ = events.muonDZ[cut]
    events.muonLooseId = events.muonLooseId[cut]
    events.muonTightId = events.muonTightId[cut]

    return events[ak.any(cut, axis = 1)]

def muonMaker(events):
    muoncut = abs(events.lepPdgId == 13)
    events['muonE'] = events.lepE[muoncut]
    events['muonPt'] = events.lepPt[muoncut]
    events['muonEta'] = events.lepEta[muoncut]
    events['muonPhi'] = events.lepPhi[muoncut]
    events['muonPdgId'] = events.lepPdgId[muoncut]
    events['muonDZ'] = events.lepDZ[muoncut]
    events['muonLooseId'] = events.lepLooseId[muoncut]
    events['muonTightId'] = events.lepTightId[muoncut]

def muon_acc_cut(events):
    cut = (ak.count(events.muonPhi, axis = 1) > 0) 
    print(ak.where(ak.count(events.muonPhi, axis = 1) == 0))
    return events[cut]

def lead_muon_cut(events):
    cut = ak.max(events.muonPt, 1) == events.muonPt
    return muonCutter(events, cut)

muonMaker(signalevents)

signalevents = muon_acc_cut(signalevents)
signalevents = lead_muon_cut(signalevents)


def deltaRPhiEta_muon_cls( events):
    clusts = ak.zip(
        {
            'pt': ak.zeros_like(events.cscRechitClusterPhi),
            'phi': events.cscRechitClusterPhi,
            'eta': events.cscRechitClusterEta,
            'E': ak.zeros_like(events.cscRechitClusterPhi),
        },
        with_name = 'PtEtaPhiMLorentzVector',
        behavior = vector.behavior,                 
      )

    muoncut = abs(events.lepPdgId) == 13
    muons = ak.zip(
        {
            'pt': events.lepPt[muoncut],
            'phi': events.lepPhi[muoncut],
            'eta': events.lepEta[muoncut],
            'E': events.lepE[muoncut],
        },
        with_name = 'PtEtaPhiMLorentzVector',
        behavior = vector.behavior,                 
      )
    leadcut = ak.max(muons.pt, 1) == muons.pt
    temp = muons
    muons = muons[leadcut]

    muon_cls_pairs = ak.cartesian({
        "clusts": clusts, 
        "muons": muons
    })

    deltaR = (muon_cls_pairs.clusts).delta_r(muon_cls_pairs.muons)
    deltaEta = abs(muon_cls_pairs.clusts.eta - muon_cls_pairs.muons.eta)
    deltaPhi = np.arctan2(np.sin(muon_cls_pairs.clusts.phi - muon_cls_pairs.muons.phi), 
                          np.cos(muon_cls_pairs.clusts.phi - muon_cls_pairs.muons.phi))


    for i in range(len(deltaR)):
        if i in [4579,15785,19586,26798,28666]:
            print('in',temp.phi[i], deltaR[i], events.cscRechitClusterZ[i])
        if i in range(25):
            print('out',temp.phi[i], deltaR[i], events.cscRechitClusterZ[i])

    #return deltaR, deltaPhi, deltaEta



deltaRPhiEta_muon_cls(signalevents)

(<Array [7, 8, 10, ... 823772, 823778, 823784] type='89942 * int64'>,)
out [1.1, -0.283, 0.776, -2.19, -1.36] [] []
out [2.62, -1.54, 1.31, 0.864, 2.34, -1.99, 1.68, 3.13, -1.34, 0.544, 1.17, -0.392] [] []
out [-2.31, -0.118, 0.324, 1.12, -0.745, -0.08] [] []
out [1.2, 1.6, 2.94, -3.08, -2.02] [] []
out [-1.58, -0.321, -0.773, 2.8, 0.606] [] []
out [3.02, 1.47] [] []
out [1.54, -3.13, -1.1, 1.2, -0.434, -0.103] [] []
out [-0.145, -0.713] [] []
out [-0.0315, -0.0974, -0.474, -0.752, -3.03] [] []
out [2.25, -2.35, -2.35] [] []
out [2.73, 2.66, -1.56, 1.06, 1.92, 0.776, 1.04] [] []
out [-1.08, 2.79, 0.956, 2.6, -0.742, -2.2, 2.35] [] []
out [-0.413, 0.295] [] []
out [0.981, 2.15, 0.476, 1.66, 3.07] [] []
out [-1.17, 3.09, -0.6, 0.757, -2.96] [] []
out [1.3, 1.09, 2.27, -2.51, -2.51, -0.509, 0.256, 0.685] [] []
out [1.15, -2.72, -0.0653, -2.11, 2.15] [] []
out [1.64, 2.04, 2.97, -1.46, -2.48] [] []
out [0.227, 0.276] [] []
out [2.35, 1.84, 3.11] [] []
out [2.31, -1.57, -0.166, -1.95] [] []

In [17]:
ind = []

lepPt = signalevents.lepPt
lepPdgId = signalevents.lepPdgId
newlepPt = lepPt[abs(lepPdgId) == 13]
for i in range(len(newlepPt)):
    if len(newlepPt[i]) == 0:
        ind.append(i)

print(ind)

[118, 194, 352, 356, 384, 517, 797, 881, 902, 1038, 1053, 1104, 1155, 1159, 1196, 1248, 1258, 1275, 1290, 1446, 1532, 1622, 1693, 1869, 1891, 1937, 1996, 2014, 2105, 2180, 2246, 2287, 2321, 2484, 2717, 2720, 2723, 2817, 2829, 2863, 2901, 3038, 3097, 3106, 3238, 3279, 3308, 3313, 3318, 3329, 3382, 3534, 3556, 3587, 3627, 3664, 3728, 4273, 4274, 4530, 4579, 4662, 4683, 4705, 4752, 4882, 4910, 4997, 5057, 5098, 5127, 5214, 5268, 5490, 5523, 5693, 5758, 5787, 5825, 5838, 5858, 5873, 5917, 6075, 6126, 6198, 6242, 6512, 6519, 6534, 6559, 6669, 6779, 6876, 7088, 7188, 7277, 7286, 7296, 7313, 7316, 7398, 7445, 7903, 8206, 8218, 8240, 8408, 8533, 8595, 8681, 8711, 8762, 8840, 8933, 8972, 9191, 9408, 9722, 9734, 9826, 9846, 9934, 9936, 10017, 10038, 10066, 10160, 10304, 10312, 10375, 10426, 10642, 10776, 10779, 10798, 10803, 10879, 11001, 11025, 11061, 11091, 11288, 11430, 11521, 11614, 11798, 11811, 11825, 11868, 11893, 11977, 11988, 11998, 12089, 12112, 12221, 12265, 12267, 12294, 12363, 12423

In [34]:
sig = set(print(f'events.{field} = events.{field}[cut]') for field in signalevents.fields if field.startswith('lep'))
bac = set(f'events.{field} = events.{field}[cut]' for field in signalevents.fields if field.startswith('lep'))

events.lepE = events.lepE[cut]
events.lepPt = events.lepPt[cut]
events.lepEta = events.lepEta[cut]
events.lepPhi = events.lepPhi[cut]
events.lepPdgId = events.lepPdgId[cut]
events.lepDZ = events.lepDZ[cut]
events.lepLooseId = events.lepLooseId[cut]
events.lepTightId = events.lepTightId[cut]


In [48]:
signalevents = NanoEventsFactory.from_root(
    signalfile,
    treepath='MuonSystem',
    #entry_stop=10,
    #metadata={"dataset": "llp"},
    schemaclass=BaseSchema,
).events()

def muonCutter(events, cut): # i hate this.
    events.muonE = events.muonE[cut]
    events.muonPt = events.muonPt[cut]
    events.muonEta = events.muonEta[cut]
    events.muonPhi = events.muonPhi[cut]
    events.muonPdgId = events.muonPdgId[cut]
    events.muonDZ = events.muonDZ[cut]
    events.muonLooseId = events.muonLooseId[cut]
    events.muonTightId = events.muonTightId[cut]

    return events[ak.any(cut, axis = 1)]

def muonMaker(events):
    muoncut = abs(events.lepPdgId == 13)
    events['muonE'] = events.lepE[muoncut]
    events['muonPt'] = events.lepPt[muoncut]
    events['muonEta'] = events.lepEta[muoncut]
    events['muonPhi'] = events.lepPhi[muoncut]
    events['muonPdgId'] = events.lepPdgId[muoncut]
    events['muonDZ'] = events.lepDZ[muoncut]
    events['muonLooseId'] = events.lepLooseId[muoncut]
    events['muonTightId'] = events.lepTightId[muoncut]

def muon_acc_cut(events):
    cut = (ak.count(events.muonPhi, axis = 1) > 0) 
    print(ak.where(ak.count(events.muonPhi, axis = 1) == 0))
    return events[cut]

def lead_muon_cut(events):
    cut = ak.max(events.muonPt, 1) == events.muonPt
    return muonCutter(events, cut)

muonMaker(signalevents)

signalevents = muon_acc_cut(signalevents)
signalevents = lead_muon_cut(signalevents)

(<Array [7, 8, 10, ... 823772, 823778, 823784] type='89942 * int64'>,)


In [47]:
ind = []

lepPt = signalevents.lepPt
lepPdgId = signalevents.lepPdgId
newlepPt = lepPt[abs(lepPdgId) == 13]
for i in range(len(newlepPt)):
    if len(newlepPt[i]) == 0:
        ind.append(i)

print(ind)

[]
